In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

from scipy.sparse import hstack

### Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [2]:
data = pd.read_csv('salary-train.csv')

In [3]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


### Проведите предобработку:

+ <b>Приведите тексты к нижнему регистру</b>
```python
(text.lower()).
```

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
FullDescription       60000 non-null object
LocationNormalized    60000 non-null object
ContractTime          44418 non-null object
SalaryNormalized      60000 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [5]:
cat_features = ['FullDescription', 'LocationNormalized', 'ContractTime']

In [6]:
for feature in cat_features:
    data[feature] = list(map(lambda x: x if pd.isnull(x) else x.lower(), data[feature]))

+ <b>Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text).</b>

In [7]:
for feature in cat_features:
    data[feature] = data[feature].replace('[^a-zA-Z0-9]', ' ', regex = True)

+ <b>Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).</b>

In [8]:
vectorizer = TfidfVectorizer(min_df=5)

In [9]:
X_FullDescription = vectorizer.fit_transform(data['FullDescription'])

In [10]:
X_FullDescription.shape

(60000, 22861)

+ <b>Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.</b>

In [11]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [12]:
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,nan,20355


+ <b>Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.</b>

In [13]:
enc = DictVectorizer()

In [14]:
X_train_cat = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [15]:
X_train_cat

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

+ <b>Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.</b>

In [16]:
X = hstack([X_train_cat, X_FullDescription])

### Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [17]:
y = data['SalaryNormalized']

In [18]:
clf = Ridge(alpha=1, random_state=241)

In [19]:
clf.fit(X, y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

### Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел

In [20]:
data_test = pd.read_csv('salary-test-mini.csv')

In [22]:
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [23]:
X_test_FullDescription = vectorizer.transform(data_test['FullDescription'])

In [24]:
X_test_cat =  enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [25]:
X_test = hstack([X_test_cat, X_test_FullDescription])

In [26]:
y_pred = clf.predict(X_test)

In [27]:
y_pred

array([ 55850.5934512 ,  37182.51069003])

In [28]:
with open('output.txt', 'w') as f:
    f.write(' '.join([str(x) for x in y_pred]))